# 光流估计

### 光流是空间运动物体在观测成像平面上的像素运动的“瞬时速度”，根据各个像素点的速度矢量特征，可以对图像进行动态分析，例如目标跟踪
    1. 亮度恒定： 同一点随着时间的变化，其亮度不会发生改变（帧和帧之间差距较小，因此一般不会发生变化）
    2. 小运动： 随着时间的变化不会引起位置的剧烈变化，只有小运动情况下才能用前后帧之间单位位置变化引起的灰度变化去近似灰度对位置的偏导数
    3. 空间一致： 一个场景上临近的点投影到图像上也是临近点，且临近点速度一直。
### 因为光流法基本方程约束只有一个，而如果要求X,Y方向的速度，有两个未知量，因此需要联立n多个方程求解

## Lucas-Kanade 算法
![](pic/1.png)
![](pic/2.png)
![](pic/3.png)
#### 注：一般情况下角点矩阵一般可逆，即λ1λ2都较大的情况下，是可逆的
### 所以光流检测中一般围绕着角点进行检测 先进性角点检测，把角点传入光流估计算法

In [17]:
import numpy as np
import cv2
cap = cv2.VideoCapture("./video/test.avi")
_,frame = cap.read()
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
# 初始化角点检测参数
feature_param = dict(
    maxCorners = 100,
    qualityLevel = 0.3,
    minDistance = 7
)
#Lucas_Kanade参数
lk_param = dict(
    winSize = (15,15),
    maxLevel = 2
)

color = np.random.randint(0,255,(100,3))

# 返回所有检测特征点，需要输入图像
p0 = cv2.goodFeaturesToTrack(gray,mask=None,**feature_param)

In [18]:
p0 # 图像中的角点
for i in p0:
    frame[int(i[0][1]),int(i[0][0]),0] = 0
    frame[int(i[0][1]),int(i[0][0]),1] = 0
    frame[int(i[0][1]),int(i[0][0]),2] = 255
cv2.imshow("frame",frame)
cv2.waitKey(0)

32

In [19]:
cv2.destroyAllWindows()

In [20]:
mask = np.zeros_like(frame)
while _:
    _,frame = cap.read()
    gray_now = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    # 光流检测
    p1, st ,err = cv2.calcOpticalFlowPyrLK(gray,gray_now,p0,None,**lk_param)

    good_mew = p1[st==1]
    good_old = p0[st==1]

    for i,(new,old) in enumerate(zip(good_mew,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask,(a,b),(c,d),color[i].tolist(),2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow("frame",img)
    k = cv2.waitKey(100) & 0xff
    if k ==27:
        break
    gray = gray_now.copy()
    p0 = good_mew.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()